In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df_combined = pd.read_json('/allah/freqtrade/json_dict/2023-09-10_16-54-32_MacdStrategyCombined.json')

FileNotFoundError: File /allah/freqtrade/json_dict/2023-09-10_16-54-32_MacdStrategyCombined.json does not exist

In [2]:
""" 30 Volume Version """
import numpy as np

lag_features = ['volume']  # List of features to create lag columns for
lag_columns = []  # List to store the names of generated lag columns

for feature in lag_features:
    for i in range(1, 21):
        df_combined[f'{feature}_{i}'] = df_combined[feature].shift(i)
        # Append the generated lag column names to the lag_columns list
        lag_columns.append(f'{feature}_{i}')

# Now, lag_columns contains the names of all generated lag columns
# df_long['macd_direction'] = np.where(df_long.macdhist > 0, 1, -1)


In [5]:
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from collections import Counter
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping

# Assuming you have already loaded your data
df_long = df_combined[(df_combined.label == 'true_long') | (df_combined.label == 'false_long')]
# Step 1: Select features and target variable
features = lag_columns
target = 'label'

# Encode the labels using LabelEncoder
label_encoder = LabelEncoder()
train_df = df_long[df_long.trade == 1.0]
train_df['label'] = label_encoder.fit_transform(train_df['label'])

# Step 2: Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    train_df[features], train_df[target], test_size=0.2,shuffle=False, random_state=42
)
# X_train, X_test, y_train, y_test = train_test_split(
#     train_df[features], train_df[target], test_size=0.2, random_state=42
# )

# Step 3: Scale the features (MinMaxScaler)
scaler = MinMaxScaler()
X_train_scaled, X_test_scaled = scaler.fit_transform(X_train), scaler.transform(X_test)

# Count the number of samples in each class
print("Class distribution before oversampling:", Counter(y_train))

# Apply SMOTE (Synthetic Minority Over-sampling Technique)
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_scaled, y_train)

# Count the number of samples in each class after oversampling
print("Class distribution after oversampling:", Counter(y_train_resampled))
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
# Step 4: Define the neural network model
model = Sequential([
    Dense(60, input_dim=len(features), activation='relu'),
    Dense(30, activation='relu'),
    Dense(3, activation='relu'),
    Dense(1, activation='sigmoid')  # Binary classification, so use 'sigmoid' activation
])

# Specify a higher learning rate for the Adam optimizer
learning_rate = 0.001  # You can adjust this value
optimizer = Adam(learning_rate=learning_rate)

# Compile the model with the specified optimizer
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# Step 5: Train the model with TensorBoard
early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1, restore_best_weights=True)

tensorboard = TensorBoard(log_dir='./logs', histogram_freq=1, write_graph=True, write_images=True)
model.fit(X_train_resampled, y_train_resampled, epochs=5000, batch_size=32, validation_data=(X_test_scaled, y_test), callbacks=[])

# Step 6: Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test_scaled, y_test)
print(f'Test Loss: {test_loss:.4f}')
print(f'Test Accuracy: {test_accuracy:.4f}')

# Inverse transform the encoded labels to get the original labels
# original_labels = label_encoder.inverse_transform(model.predict(X_test_scaled) > 0.5)


Class distribution before oversampling: Counter({0: 16434, 1: 5197})
Class distribution after oversampling: Counter({1: 16434, 0: 16434})
X_train shape: (21631, 20)
y_train shape: (21631,)
Epoch 1/5000
1028/1028 [==============================] - 2s 1ms/step - loss: 0.6803 - accuracy: 0.5591 - val_loss: 0.6877 - val_accuracy: 0.4832
Epoch 2/5000
1028/1028 [==============================] - 1s 1ms/step - loss: 0.6758 - accuracy: 0.5713 - val_loss: 0.6926 - val_accuracy: 0.5050
Epoch 3/5000
1028/1028 [==============================] - 1s 1ms/step - loss: 0.6748 - accuracy: 0.5700 - val_loss: 0.6917 - val_accuracy: 0.5146
Epoch 4/5000
1028/1028 [==============================] - 1s 1ms/step - loss: 0.6739 - accuracy: 0.5736 - val_loss: 0.6903 - val_accuracy: 0.5009
Epoch 5/5000
 698/1028 [===================>..........] - ETA: 0s - loss: 0.6735 - accuracy: 0.5723

KeyboardInterrupt: 

In [ ]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from collections import Counter

# Assuming you have already loaded your data
# df_long = df_combined[(df_combined.label == 'true_long') | (df_combined.label == 'false_long')]

# Step 1: Select features and target variable
features = lag_columns
target = 'label'

# Encode the labels using LabelEncoder
label_encoder = LabelEncoder()
# train_df = df_long[df_long.trade == 1.0]
# Assuming you have already loaded your data and encoded labels
# train_df['label'] = label_encoder.fit_transform(train_df['label'])

# Sample data (replace this with your actual data)
X_train, y_train, X_test, y_test = np.random.rand(100, len(features)), np.random.randint(0, 2, 100), np.random.rand(20, len(features)), np.random.randint(0, 2, 20)

# Step 2: Scale the features (MinMaxScaler)
scaler = MinMaxScaler()
X_train_scaled, X_test_scaled = scaler.fit_transform(X_train), scaler.transform(X_test)

# Count the number of samples in each class
print("Class distribution before oversampling:", Counter(y_train))

# Apply SMOTE (Synthetic Minority Over-sampling Technique)
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_scaled, y_train)

# Count the number of samples in each class after oversampling
print("Class distribution after oversampling:", Counter(y_train_resampled))
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)

# Step 3: Define a simple neural network
class NeuralNetwork:
    def __init__(self):
        self.weights = None
        self.bias = None

    def sigmoid(self, z):
        return 1 / (1 + np.exp(-z))

    def initialize_params(self, num_features):
        self.weights = np.zeros(num_features)
        self.bias = 0

    def forward(self, X):
        return self.sigmoid(np.dot(X, self.weights) + self.bias)

    def compute_loss(self, y, y_pred):
        m = len(y)
        return -1/m * np.sum(y * np.log(y_pred) + (1 - y) * np.log(1 - y_pred))

    def fit(self, X, y, learning_rate, num_epochs):
        m, num_features = X.shape
        self.initialize_params(num_features)

        for epoch in range(num_epochs):
            y_pred = self.forward(X)

            dw = (1/m) * np.dot(X.T, (y_pred - y))
            db = (1/m) * np.sum(y_pred - y)

            self.weights -= learning_rate * dw
            self.bias -= learning_rate * db

            loss = self.compute_loss(y, y_pred)
            print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss:.4f}")

    def predict(self, X):
        y_pred = self.forward(X)
        return (y_pred > 0.5).astype(int)

# Step 4: Train the simple neural network
learning_rate = 0.001
num_epochs = 5000

model = NeuralNetwork()
model.fit(X_train_resampled, y_train_resampled, learning_rate, num_epochs)

# Step 5: Evaluate the model
y_pred_test = model.predict(X_test_scaled)
accuracy = np.mean(y_pred_test == y_test)
print(f'Test Accuracy: {accuracy:.4f}')
